In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

train = pd.read_csv('/content/gdrive/MyDrive/데이콘/open/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/데이콘/open/test.csv')
sub = pd.read_csv('/content/gdrive/MyDrive/데이콘/open/sample_submission.csv')
snp_info = pd.read_csv('/content/gdrive/MyDrive/데이콘/open/snp_info.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
train.shape

(262, 21)

In [4]:
snp_list = ["SNP_01","SNP_02","SNP_03","SNP_04","SNP_05","SNP_06","SNP_07","SNP_08","SNP_09","SNP_10","SNP_11","SNP_12","SNP_13","SNP_14","SNP_15"]

In [5]:
train = train.drop(columns = ["father","mother","gender"], axis =1)
test = test.drop(columns = ["father","mother","gender"], axis =1)

In [6]:
for snp in snp_list:
  train = pd.concat([train,pd.get_dummies(train[snp], [snp])], axis = 1)
  train = train.drop(columns = [snp], axis =1)
  test = pd.concat([test,pd.get_dummies(test[snp], [snp])], axis = 1)
  test = test.drop(columns = [snp], axis =1)


In [7]:
train["class"] = train["class"].map({'A': 0,'B':1, 'C':2})

In [13]:
test

,id,trait,['SNP_01']_A A,['SNP_01']_A G,['SNP_01']_G G,['SNP_02']_A A,['SNP_02']_A G,['SNP_02']_G G,['SNP_03']_A A,['SNP_03']_C A,...,['SNP_12']_G G,['SNP_13']_A A,['SNP_13']_A G,['SNP_13']_G G,['SNP_14']_A A,['SNP_14']_C A,['SNP_14']_C C,['SNP_15']_A A,['SNP_15']_G A,['SNP_15']_G G
0,TEST_000,1,0,1,0,0,0,1,1,0,...,0,0,0,1,0,1,0,0,1,0
1,TEST_001,2,0,0,1,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,TEST_002,2,0,0,1,0,1,0,1,0,...,0,0,0,1,1,0,0,0,0,1
3,TEST_003,2,0,0,1,0,1,0,0,1,...,0,0,1,0,1,0,0,1,0,0
4,TEST_004,1,1,0,0,0,0,1,1,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,TEST_170,2,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
171,TEST_171,2,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,0,1,0
172,TEST_172,2,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,1
173,TEST_173,2,0,1,0,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0


In [14]:
X_train = train.drop(columns=["class", "id"], axis = 1)
y_train = train["class"]

X_test = test.drop(columns = ["id"])

In [12]:
X_train

,id,trait,['SNP_01']_A A,['SNP_01']_A G,['SNP_01']_G G,['SNP_02']_A A,['SNP_02']_A G,['SNP_02']_G G,['SNP_03']_A A,['SNP_03']_C A,...,['SNP_12']_G G,['SNP_13']_A A,['SNP_13']_A G,['SNP_13']_G G,['SNP_14']_A A,['SNP_14']_C A,['SNP_14']_C C,['SNP_15']_A A,['SNP_15']_G A,['SNP_15']_G G
0,TRAIN_000,2,0,0,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
1,TRAIN_001,2,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,0
2,TRAIN_002,2,0,0,1,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0
3,TRAIN_003,1,1,0,0,0,0,1,1,0,...,1,0,0,1,1,0,0,0,0,1
4,TRAIN_004,2,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,TRAIN_257,2,0,1,0,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
258,TRAIN_258,2,0,0,1,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
259,TRAIN_259,1,0,1,0,0,0,1,1,0,...,1,0,0,1,0,1,0,0,0,1
260,TRAIN_260,1,1,0,0,0,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1


In [15]:
import lightgbm 

r = 0
classifiers = {"kNN":KNeighborsClassifier(), 
               "SVC":SVC(kernel="linear"),
               "DTC":DecisionTreeClassifier(random_state=r) ,
               "RFC":RandomForestClassifier(random_state=r) ,
               "GNB":GaussianNB(),
               "MLP":MLPClassifier(random_state=r),
               "XGB":XGBClassifier(random_state=r)}

for name in classifiers: 
    score = cross_val_score(classifiers[name], X_train, y_train, cv=5).mean()
    print(name, score.round(4))   

kNN 0.9238
SVC 0.9391
DTC 0.9087
RFC 0.939
GNB 0.901


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP 0.9468
XGB nan


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py", line 725, in fit
    trai

In [19]:
import tensorflow.keras as keras
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold


final_model = keras.models.Sequential([
    keras.layers.Dense(46,activation = 'relu', input_dim = 46),
    keras.layers.Dense(24,activation = 'relu'),
    keras.layers.Dense(8,activation = 'relu'),
    keras.layers.Dense(3, activation = 'softmax')
])

final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
from keras.utils import np_utils
Y_encoded = np_utils.to_categorical(y_train)

final_model.fit(X_train,Y_encoded,epochs=30, batch_size=1)

Epoch 1/30
262/262 [==============================] - 1s 2ms/step - loss: 0.7333 - acc: 0.7366
Epoch 2/30
262/262 [==============================] - 0s 2ms/step - loss: 0.2064 - acc: 0.9389
Epoch 3/30
262/262 [==============================] - 0s 2ms/step - loss: 0.1051 - acc: 0.9618
Epoch 4/30
262/262 [==============================] - 0s 2ms/step - loss: 0.1195 - acc: 0.9542
Epoch 5/30
262/262 [==============================] - 0s 2ms/step - loss: 0.0595 - acc: 0.9771
Epoch 6/30
262/262 [==============================] - 0s 2ms/step - loss: 0.0719 - acc: 0.9695
Epoch 7/30
262/262 [==============================] - 0s 2ms/step - loss: 0.0470 - acc: 0.9847
Epoch 8/30
262/262 [==============================] - 0s 2ms/step - loss: 0.0426 - acc: 0.9885
Epoch 9/30
262/262 [==============================] - 0s 2ms/step - loss: 0.0286 - acc: 0.9885
Epoch 10/30
262/262 [==============================] - 0s 2ms/step - loss: 0.0368 - acc: 0.9847
Epoch 11/30
262/262 [============================

In [33]:
y_pred = final_model.predict(X_test)

y_pred = y_pred.argmax(axis=1)

6/6 [==============================] - 0s 2ms/step


In [34]:
y_pred

array([0, 1, 2, 1, 0, 1, 2, 1, 0, 0, 2, 1, 2, 0, 1, 1, 0, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 1, 2, 0, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1,
       2, 0, 1, 1, 2, 1, 1, 2, 0, 1, 0, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1,
       2, 1, 1, 1, 1, 0, 0, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 2, 0, 1, 2, 2, 1, 0, 0, 2, 1, 1, 0, 1, 2, 2, 1, 1])

In [35]:
y_pred = np.where(y_pred == 0, "A", y_pred)
y_pred = np.where(y_pred == "1", "B", y_pred)
y_pred = np.where(y_pred == "2", "C", y_pred)

In [36]:
y_pred

array(['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'C',
       'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B',
       'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B',
       'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B',
       'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B',
       'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B',
       'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C',
       'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A',
       'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'B', 'B', 'A', 'B',
       'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C',
       'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A',
       'A', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'A', 'A', 'C', 'B', 'B',
       'A', 'B', 'C', 'C', 'B', 'B'], dtype='<U21')

In [37]:
sub["class"] = y_pred # 이전 셀에서 최종 모델로 생성한 답안

sub.to_csv(f"2020122025.csv", index=False) # 생성된 csv 파일을 좌측 폴더에서 다운로드하여 제출